In [2]:
#Library Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import warnings

from collections import Counter
warnings.filterwarnings('ignore')

In [3]:
#Data Imports

#Book Info
books = pd.read_csv("../data/raw_data/books.csv")
print(books.shape)
print(books.columns)

print()
#User ratings
ratings = pd.read_csv("../data/raw_data/ratings.csv")
print(ratings.shape)
print(ratings.columns)

# #User Book viewing history
# toRead = pd.read_csv("goodbooks-10k/data/raw_data/to_read.csv")
# print(toRead.columns)


(10000, 23)
Index(['book_id', 'goodreads_book_id', 'best_book_id', 'work_id',
       'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year',
       'original_title', 'title', 'language_code', 'average_rating',
       'ratings_count', 'work_ratings_count', 'work_text_reviews_count',
       'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5',
       'image_url', 'small_image_url'],
      dtype='object')

(5976479, 3)
Index(['user_id', 'book_id', 'rating'], dtype='object')


In [4]:
books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [5]:
ratings.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


## Books Cleaning

In [6]:
#From Books table we drop some of the columns we don't see direct use for
drop_cols =['best_book_id', 'work_id','isbn', 'isbn13', 'image_url', 'small_image_url']
books.drop(drop_cols, axis=1, inplace=True)
print(books.columns)


books.head()


Index(['book_id', 'goodreads_book_id', 'books_count', 'authors',
       'original_publication_year', 'original_title', 'title', 'language_code',
       'average_rating', 'ratings_count', 'work_ratings_count',
       'work_text_reviews_count', 'ratings_1', 'ratings_2', 'ratings_3',
       'ratings_4', 'ratings_5'],
      dtype='object')


,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
0,1,2767052,272,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317
1,2,3,491,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543
2,3,41865,226,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439
3,4,2657,487,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267
4,5,4671,1356,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718


In [7]:
#Drop all NaN rows
books.dropna(subset=['language_code'])

#Check what type of languages are covered in the books
lang_codes = np.array(list(books["language_code"]))
u_langs = np.unique(lang_codes)

print(len(lang_codes))
print(len(u_langs))
u_langs

10000
26


array(['ara', 'dan', 'en', 'en-CA', 'en-GB', 'en-US', 'eng', 'fil', 'fre',
       'ger', 'ind', 'ita', 'jpn', 'mul', 'nan', 'nl', 'nor', 'per',
       'pol', 'por', 'rum', 'rus', 'spa', 'swe', 'tur', 'vie'],
      dtype='<U5')

### Here we see that there are many other language types.

After going through the data (won't clutter your screen) we can see that these books are out of scope for our recommender system as the books themselves classsify in a different way. Because of the cultural differences, and literal variance between english books and those of other languages, we deem non-english books out of scope

In [7]:
books[books["language_code"] == 'jpn'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
2519,2520,6443834,76,"Haruki Murakami, 村上 春樹",2009.0,1Q84 Book 1 [Ichi-kyū-hachi-yon],"1Q84 BOOK 1 (1Q84, #1)",jpn,3.96,32323,36769,1280,1325,2014,6601,13628,13201
3265,3266,4634266,14,Masashi Kishimoto,2008.0,NARUTO -ナルト- 巻ノ四十三,NARUTO -ナルト- 巻ノ四十三,jpn,4.57,26364,26971,147,259,480,2170,4863,19199
4414,4415,1429082,21,"Naoko Takeuchi, 武内 直子",1991.0,美少女戦士セーラームーン,美少女戦士セーラームーン新装版 1 [Bishōjo Senshi Sailor Moon ...,jpn,4.28,18035,22319,724,664,866,2981,4933,12875
4929,4930,1717890,14,"Jun Mochizuki, 望月淳",2006.0,Pandora Hearts 1,Pandora Hearts 1巻,jpn,4.20,19222,20167,394,470,1024,3142,4899,10632
6119,6120,7717482,112,"Haruki Murakami, 村上 春樹",2010.0,1Q84 Book 3 [Ichi-kyū-hachi-yon],"1Q84 BOOK 3 (1Q84, #3)",jpn,3.86,10317,17189,1332,328,1253,4136,6260,5212


In [8]:
books[books["language_code"] == 'mul'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
8818,8819,46201,11,"Rainer Maria Rilke, Stephen Mitchell, Robert Hass",1976.0,NaN,The Selected Poetry of Rainer Maria Rilke,mul,4.4,11223,11342,248,111,187,1179,3435,6430


In [9]:
books[books["language_code"] == 'nl'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
1090,1091,85379,105,Patricia Cornwell,1993.0,Cruel & Unusual,"Cruel & Unusual (Kay Scarpetta, #4)",nl,4.11,83695,86940,690,586,1978,17291,34226,32859


In [10]:
books[books["language_code"] == 'nor'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
2430,2431,1991039,139,"Jo Nesbø, Don Bartlett",1997.0,Flaggermusmannen,"Flaggermusmannen (Harry Hole, #1)",nor,3.56,22230,48143,3957,1478,4686,16180,16807,8992
3646,3647,6891433,117,Jo Nesbø,2009.0,"Panserhjerte (Harry Hole, #8)","Panserhjerte (Harry Hole, #8)",nor,4.08,10991,30903,2370,227,799,5498,14278,10101
4968,4969,7147831,93,Karl Ove Knausgård,2009.0,Min kamp 1,Min kamp 1 (Min kamp #1),nor,4.08,7886,20208,2345,373,956,3151,7999,7729


In [11]:
books[books["language_code"] == 'per'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
6159,6160,326272,127,"Heinrich Böll, محمد اسماعیل‌زاده",1963.0,Ansichten eines Clowns,عقاید یک دلقک,per,4.05,4225,13897,824,225,659,2598,5101,5314
6771,6772,716696,777,"Omar Khayyám, محمدعلی فروغی, قاسم غنی",1120.0,رباعیات [Robāʿiāt-e ʿOmar Khayyām],رباعيات خيام,per,4.18,3200,13724,1111,218,568,2233,4163,6542
7484,7485,364826,155,"Mika Waltari, ذبیح‌الله منصوری",1945.0,Sinuhe egyptiläinen,سینوهه,per,4.08,5907,11871,545,344,641,1958,3743,5185
7638,7639,192510,91,"José Mauro de Vasconcelos, قاسم صنعوی",1968.0,Meu Pé de Laranja Lima,درخت زیبای من,per,4.36,2716,15091,931,164,392,1644,4516,8375
8335,8336,74862,43,Samad Behrangi,1965.0,ماهی سیاه کوچولو,ماهی سیاه کوچولو,per,4.11,5364,8080,395,127,387,1465,2562,3539


In [12]:
books[books["language_code"] == 'pol'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
3995,3996,68458,96,Andrzej Sapkowski,1994.0,Krew elfów,"Krew elfów (Saga o Wiedźminie, #3)",pol,4.23,11981,29790,1135,152,709,4315,11554,13060
4485,4486,1128437,90,Andrzej Sapkowski,1993.0,Miecz przeznaczenia,"Miecz przeznaczenia (Saga o Wiedźminie, #2)",pol,4.40,12909,26446,1008,78,348,2618,9245,14157
5248,5249,68453,78,Andrzej Sapkowski,1995.0,Czas pogardy,"Czas pogardy (Saga o Wiedźminie, #4)",pol,4.32,11551,22876,584,68,377,2815,8520,11096
6057,6058,1876770,69,Andrzej Sapkowski,1996.0,Chrzest ognia,"Chrzest ognia (Saga o Wiedźminie, #5)",pol,4.36,10840,19382,517,47,325,2176,6812,10022
7068,7069,68455,63,Andrzej Sapkowski,1997.0,Wieża Jaskółki,"Wieża Jaskółki (Saga o Wiedźminie, #6)",pol,4.34,10195,16630,400,73,347,1955,5744,8511


In [13]:
books[books["language_code"] == 'rum'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
8811,8812,817199,30,Mircea Eliade,1933.0,Maitreyi,Maitreyi,rum,3.92,11079,11899,184,248,817,2702,3998,4134


In [14]:
books[books["language_code"] == 'rus'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
9883,9884,97390,32,"Aleksandr Griboyedov, Александр Сергеевич Гриб...",1825.0,Горе от ума,Горе от ума,rus,4.13,9287,9690,36,159,553,1674,2791,4513


In [15]:
books[books["language_code"] == 'spa'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
47,48,4381,507,Ray Bradbury,1953.0,Fahrenheit 451,Fahrenheit 451,spa,3.97,570498,1176240,30694,28366,64289,238242,426292,419051
83,84,7677,176,Michael Crichton,1990.0,Jurassic Park,"Jurassic Park (Jurassic Park, #1)",spa,3.96,447833,650483,8143,15736,34222,138327,231583,230615
555,556,10603,143,Stephen King,1981.0,Cujo,Cujo,spa,3.65,158215,167044,2832,4096,16130,54393,52708,39717
914,915,10585,125,"Stephen King, Bettina Blanch Tyroller",1994.0,Insomnia,Insomnia,spa,3.79,100972,109444,2296,3138,9066,28408,35608,33224
1799,1800,22590,151,"Philip K. Dick, David Alabort, Manuel Espín",1969.0,Ubik,Ubik,spa,4.10,48256,58063,2957,548,2176,10594,22241,22504


In [16]:
books[books["language_code"] == 'swe'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
7751,7752,1457790,100,Camilla Läckberg,2005.0,Stenhuggaren,"Stenhuggaren (Patrik Hedström, #3)",swe,3.85,6435,15722,1269,260,759,3862,7092,3749


In [17]:
books[books["language_code"] == 'tur'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
4942,4943,220826,28,Sabahattin Ali,1943.0,Kürk Mantolu Madonna,Kürk Mantolu Madonna,tur,4.49,19806,21803,1277,114,348,1785,6029,13527


In [18]:
books[books["language_code"] == 'vie'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
3008,3009,81922,84,"Robert T. Kiyosaki, Sharon L. Lechter",1998.0,Cashflow Quadrant: Rich Dad's Guide to Financi...,Rich Dad's Cashflow Quadrant: Rich Dad's Guide...,vie,4.05,24907,27477,718,560,1360,5288,9087,11182


In [19]:
books[books["language_code"] == 'ara'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
1371,1372,16031620,6,أحمد مراد,2012.0,الفيل الأزرق,الفيل الأزرق,ara,3.80,53187,54340,5771,2780,4687,11444,16996,18433
1474,1475,16081961,7,"أحلام مستغانمي, Ahlam Mosteghanemi",2012.0,الأسود يليق بك,الأسود يليق بك,ara,3.72,42958,52255,5459,3323,5535,11642,13878,17877
1646,1647,7704143,13,أحمد مراد,2010.0,تراب الماس,تراب الماس,ara,4.09,43016,45608,4027,995,2050,7402,16743,18418
1786,1787,20317106,7,محمد صادق,2014.0,هيبتا,هيبتا,ara,3.82,42978,43298,5680,2886,4135,8052,11237,16988
1932,1933,7119070,11,أثير عبدالله النشمي,2010.0,أحببتك أكثر مما ينبغى,أحببتك أكثر مما ينبغي,ara,3.67,31779,42357,4898,3563,4620,8914,10505,14755


In [20]:
books[books["language_code"] == 'dan'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
5239,5240,2033352,129,Jo Nesbø,2005.0,"Frelseren (Harry Hole, #6)","The Redeemer (Harry Hole, #6)",dan,4.07,9337,21357,1561,196,514,3746,10080,6821
6871,6872,7145565,85,Jussi Adler-Olsen,2009.0,Flaskepost fra P,"Flaskepost fra P (Afdeling Q, #3)",dan,4.04,9658,18828,1347,374,606,3164,8344,6340
9443,9444,10008056,75,Jussi Adler-Olsen,2010.0,Journal 64,"Journal 64 (Afdeling Q, #4)",dan,4.04,5044,12025,995,95,316,2249,5752,3613


In [21]:
books[books["language_code"] == 'fil'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
5787,5788,1433337,3,Bob Ong,2001.0,ABNKKBSNPLAKo?! (Mga Kwentong Chalk ni Bob Ong),ABNKKBSNPLAKo?! (Mga Kwentong Chalk ni Bob Ong),fil,4.2,14481,14537,508,196,534,2343,4544,6920
9344,9345,17903381,2,"HaveYouSeenThisGirL, Jan Irene Villar",2013.0,NaN,Diary ng Panget,fil,4.4,7016,7045,401,294,297,638,891,4925


In [22]:
books[books["language_code"] == 'fre'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
577,578,10629,156,"Stephen King, Marie Milpois",1983.0,Christine,Christine,fre,3.72,151160,157664,2092,3156,14024,48479,49499,42506
787,788,23955,112,"Irvine Welsh, Eric Lindor Fall",1993.0,Trainspotting,Trainspotting,fre,4.07,59453,110677,2029,1757,4329,19778,42833,41980
817,818,11570,118,"Stephen King, William Olivier Desmond",2001.0,Dreamcatcher,Dreamcatcher,fre,3.59,115855,122599,1856,5234,14010,36926,36389,30040
1074,1075,10625,146,"Stephen King, Dominique Dill",1992.0,Dolores Claiborne,Dolores Claiborne,fre,3.81,71053,98959,1718,1623,7586,27461,33524,28765
2385,2386,9522,12,Marjane Satrapi,2000.0,NaN,"Persepolis, Volume 1",fre,4.27,46993,47133,301,1200,1409,5430,14422,24672


In [23]:
books[books["language_code"] == 'ger'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
1844,1845,58027,91,"Margaret Atwood, Brigitte Walitzek",1996.0,Alias Grace,Alias Grace,ger,3.97,51511,60149,2881,590,2339,13103,26233,17884
2096,2097,24861,347,Hermann Hesse,1919.0,Demian: Die Geschichte einer Jugend,Demian. Die Geschichte von Emil Sinclairs Jugend,ger,4.10,37459,49633,2305,580,2061,9154,17689,20149
3072,3073,6919272,68,Kerstin Gier,2010.0,Saphirblau,"Saphirblau (Edelstein-Trilogie, #2)",ger,4.19,25767,53718,4359,360,1569,8515,20563,22711
3369,3370,68811,171,Michael Ende,1973.0,Momo,Momo,ger,4.29,28857,34528,1639,337,1003,4606,11089,17493
3424,3425,16707,50,"Michael Chabon, Hans Hermann",1995.0,Wonder Boys,Wonder Boys,ger,3.93,24365,29212,1664,328,1377,6459,12814,8234


In [24]:
books[books["language_code"] == 'ita'].head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
5939,5940,19164,385,"Dante Alighieri, Robert M. Durling",1320.0,Purgatorio,"Purgatorio (The Divine Comedy, #2)",ita,4.01,13740,17533,511,210,1038,3980,5436,6869
9940,9941,61266,72,Alessandro Baricco,1994.0,Novecento,Novecento. Un monologo,ita,4.19,9991,11676,575,121,448,1784,4052,5271


In [8]:
lang_mask = [False if math.isnan(b) else b for b in books['language_code'].str.contains("en")]    
np.unique(lang_mask)

array([False,  True])

In [9]:
#Apply boolean mask 
books = books[lang_mask]

In [10]:
print(books.shape)
print(np.unique(books.language_code))
books.head()

(8730, 17)
['en' 'en-CA' 'en-GB' 'en-US' 'eng']


,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
0,1,2767052,272,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317
1,2,3,491,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543
2,3,41865,226,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439
3,4,2657,487,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267
4,5,4671,1356,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718


### As we can see we've dropped our books data by 1,270 books

### Check for missing data in books table

In [11]:
# Checks for missing value for each column
for col in books.columns:
    mis = books[col].isnull().value_counts()
    print("Column: {0}".format(col))
    print(mis)
    print()

Column: book_id
False    8730
Name: book_id, dtype: int64

Column: goodreads_book_id
False    8730
Name: goodreads_book_id, dtype: int64

Column: books_count
False    8730
Name: books_count, dtype: int64

Column: authors
False    8730
Name: authors, dtype: int64

Column: original_publication_year
False    8715
True       15
Name: original_publication_year, dtype: int64

Column: original_title
False    8234
True      496
Name: original_title, dtype: int64

Column: title
False    8730
Name: title, dtype: int64

Column: language_code
False    8730
Name: language_code, dtype: int64

Column: average_rating
False    8730
Name: average_rating, dtype: int64

Column: ratings_count
False    8730
Name: ratings_count, dtype: int64

Column: work_ratings_count
False    8730
Name: work_ratings_count, dtype: int64

Column: work_text_reviews_count
False    8730
Name: work_text_reviews_count, dtype: int64

Column: ratings_1
False    8730
Name: ratings_1, dtype: int64

Column: ratings_2
False    8730
Nam

We can verify through this that all the critical columns we care about for our recommender system all do not have any null values!

## Ratings Cleaning

In [12]:
print(ratings.shape)
ratings.head()

(5976479, 3)


,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [13]:
# Checks for missing value for each column
for col in ratings.columns:
    mis = ratings[col].isnull().value_counts()
    print("Column: {0}".format(col))
    print(mis)
    print()

Column: user_id
False    5976479
Name: user_id, dtype: int64

Column: book_id
False    5976479
Name: book_id, dtype: int64

Column: rating
False    5976479
Name: rating, dtype: int64



Looks great! no missing values!!

### Join "Books" with "Ratings" on book_id!

In [16]:
join_data = pd.merge(books, ratings, left_on='book_id', right_on='book_id', how='inner')
join_data.head()

,book_id,goodreads_book_id,books_count,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,user_id,rating
0,1,2767052,272,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,2886,5
1,1,2767052,272,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,6158,5
2,1,2767052,272,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,3991,4
3,1,2767052,272,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,5281,5
4,1,2767052,272,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,5721,5


In [33]:
# Save data to csv file
# join_data.to_csv(r'/Users/karthikrameshbabu/midsScratch/MIDS/w207/BooksFinalProject/MIDS207-Final-Project/data/parsed_data/books_join_user.csv', index = False)


In [17]:
# Filters the items that we need
rec_data = join_data.filter(items=['book_id', 'goodreads_book_id', 'title', 'average_rating', 'user_id', 'rating'])

print(rec_data.shape)
rec_data.head(20)

(5579146, 6)


,book_id,goodreads_book_id,title,average_rating,user_id,rating
0,1,2767052,"The Hunger Games (The Hunger Games, #1)",4.34,2886,5
1,1,2767052,"The Hunger Games (The Hunger Games, #1)",4.34,6158,5
2,1,2767052,"The Hunger Games (The Hunger Games, #1)",4.34,3991,4
3,1,2767052,"The Hunger Games (The Hunger Games, #1)",4.34,5281,5
4,1,2767052,"The Hunger Games (The Hunger Games, #1)",4.34,5721,5
5,1,2767052,"The Hunger Games (The Hunger Games, #1)",4.34,5034,5
6,1,2767052,"The Hunger Games (The Hunger Games, #1)",4.34,695,5
7,1,2767052,"The Hunger Games (The Hunger Games, #1)",4.34,5021,5
8,1,2767052,"The Hunger Games (The Hunger Games, #1)",4.34,3209,5
9,1,2767052,"The Hunger Games (The Hunger Games, #1)",4.34,6769,4


In [18]:
# Checks for missing value for each column
for col in rec_data.columns:
    mis = rec_data[col].isnull().value_counts()
    print("Column: {0}".format(col))
    print(mis)
    print()    
print("No NaNs!")

Column: book_id
False    5579146
Name: book_id, dtype: int64

Column: goodreads_book_id
False    5579146
Name: goodreads_book_id, dtype: int64

Column: title
False    5579146
Name: title, dtype: int64

Column: average_rating
False    5579146
Name: average_rating, dtype: int64

Column: user_id
False    5579146
Name: user_id, dtype: int64

Column: rating
False    5579146
Name: rating, dtype: int64

No NaNs!


In [19]:
#Check number of times each unique book was rated
Counter(rec_data['title'])

Counter({'The Hunger Games (The Hunger Games, #1)': 22806,
         "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)": 21850,
         'Twilight (Twilight, #1)': 16931,
         'To Kill a Mockingbird': 19088,
         'The Great Gatsby': 16604,
         'The Fault in Our Stars': 11264,
         'The Hobbit': 15558,
         'The Catcher in the Rye': 14472,
         'Angels & Demons  (Robert Langdon, #1)': 13451,
         'Pride and Prejudice': 13445,
         'The Kite Runner': 12698,
         'Divergent (Divergent, #1)': 11780,
         '1984': 14693,
         'Animal Farm': 14328,
         'The Diary of a Young Girl': 12101,
         'The Girl with the Dragon Tattoo (Millennium, #1)': 14382,
         'Catching Fire (The Hunger Games, #2)': 16549,
         'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)': 15855,
         'The Fellowship of the Ring (The Lord of the Rings, #1)': 12530,
         'Mockingjay (The Hunger Games, #3)': 15953,
         'Harry Potter an

In [20]:
# See how many times different reviewers rated the same title
rec_data.groupby("user_id")["title"].unique().head(20)

user_id
1     [To Kill a Mockingbird, Pride and Prejudice, T...
2     [Harry Potter and the Sorcerer's Stone (Harry ...
3     [To Kill a Mockingbird, 1984, The Diary of a Y...
4     [Harry Potter and the Sorcerer's Stone (Harry ...
5     [The Fault in Our Stars, The Secret Life of Be...
6     [Eat, Pray, Love, The Curious Incident of the ...
7     [The Great Gatsby, The Catcher in the Rye, 198...
8     [To Kill a Mockingbird, The Great Gatsby, Prid...
9     [The Hunger Games (The Hunger Games, #1), Harr...
10    [To Kill a Mockingbird, The Great Gatsby, Prid...
11    [To Kill a Mockingbird, The Catcher in the Rye...
12    [Harry Potter and the Sorcerer's Stone (Harry ...
13    [The Kite Runner, The Girl with the Dragon Tat...
14    [The Hunger Games (The Hunger Games, #1), The ...
15    [Harry Potter and the Sorcerer's Stone (Harry ...
16    [The Catcher in the Rye, Pride and Prejudice, ...
17    [Memoirs of a Geisha, A Time to Kill, The Firm...
18    [Twilight (Twilight, #1), To Kill 

In [21]:
# See all the different users that rated "To Kill a Mockingbird" multiple times
same_names = rec_data[rec_data['title'] == 'To Kill a Mockingbird']["user_id"].unique()
for name in same_names:
    print(name)

15
25
40
69
72
73
83
85
29
80
123
124
24
115
136
4
125
142
149
158
89
36
169
137
194
199
10
204
205
207
212
223
219
228
233
237
240
247
253
258
263
269
276
296
340
344
345
347
187
362
361
316
28
331
388
337
395
177
403
320
439
444
446
447
453
245
454
463
420
485
497
506
513
475
516
517
518
521
504
535
530
539
415
548
473
580
590
378
599
600
180
598
610
605
616
635
190
643
637
648
653
657
498
549
488
684
43
412
688
670
699
698
494
663
712
715
716
718
731
749
751
750
759
763
765
18
756
779
795
806
782
697
810
813
814
812
822
480
830
809
760
866
861
879
880
885
914
912
905
919
929
930
458
936
897
950
928
942
943
209
896
980
835
989
991
992
973
995
1002
510
1013
1016
642
926
1028
1033
966
1044
976
1042
1055
1063
1058
1074
1076
156
1084
1089
790
979
791
547
875
1122
542
1124
1030
1139
1143
452
1144
1109
1160
1104
1145
1127
1135
1112
1148
1126
1175
1176
623
1189
1193
1197
1200
1206
1147
1167
987
1220
366
1209
1236
1253
1252
1246
1219
1251
1261
1195
1265
1115
1283
1286
1293
746
1298
1238
1306

13233
13338
13341
13347
6368
12536
13351
13352
13353
5064
11
2668
13360
13361
13132
13365
13370
13374
1256
13375
13378
7076
13384
13387
13391
13399
13315
12506
13423
13425
12421
13430
13431
13434
13437
13438
13439
13447
13449
13446
13454
13455
13458
189
12742
6699
13279
13480
13482
13483
9494
13489
13146
13497
13502
13506
5563
13512
13522
13536
13540
13243
13550
13554
12899
13560
13010
13571
13472
13585
13586
13587
13592
13271
13600
13599
13596
922
13612
13614
6362
13630
13633
13635
13641
13642
13647
13651
13652
13653
13654
13655
13661
13667
7812
1997
13673
13685
13479
13674
13701
10093
13703
13704
13699
10466
13679
5471
9491
13728
11089
13725
13730
13733
13734
13662
13735
5320
13747
13748
13757
13756
878
13738
10065
13764
3836
13778
13779
13785
662
13792
13793
13794
13795
13799
13788
13802
13803
13805
13807
8314
13810
13552
8689
13191
13815
13731
13820
13830
13831
13843
13776
13848
13849
11365
4546
13863
9319
10133
13872
13159
4346
4516
13881
6605
13882
1868
13886
13890
13891
13892
13

22683
22685
22645
17957
22679
22689
22698
22700
5941
22629
16503
22705
22694
22709
22711
22713
22719
22609
22722
22726
22728
22730
22729
22732
22735
22740
22695
22742
22746
22748
22751
22750
18364
22756
22757
21989
22054
22777
22782
22781
22783
22790
22453
22795
22579
11308
22800
22804
22805
22508
22807
22809
21428
22813
22774
22817
10462
6630
22836
22835
16986
22846
22852
22854
22856
22861
22865
22867
22873
22877
22881
22755
22853
22891
22892
21601
22772
21731
22901
22905
8658
22910
2851
22906
22915
22922
22930
22932
5597
22520
22661
11328
4052
22876
22944
22949
22952
22953
22951
22859
20308
22964
22968
22972
22974
22978
22983
22996
22998
22913
23008
23011
23003
23016
23017
23019
22882
23032
23038
22487
9970
23044
19609
23050
23049
23056
23053
23062
23063
23064
23068
21380
23077
23103
11618
17799
23101
23111
17228
23125
20113
23139
23142
23150
22971
23154
23156
23158
23159
23130
23161
23163
23083
23173
23172
12781
23176
18815
23183
23190
23193
23197
19911
23199
23213
23215
23219
23145

33278
33288
33250
30923
33293
33290
21402
33312
33317
23428
33259
33330
33337
12702
33348
33354
33357
33364
33366
33356
33361
33375
33327
33385
33380
22764
10798
31601
33397
33396
5504
33172
17715
33408
33425
33427
33254
33438
23421
33457
33032
33465
33458
33466
33464
29001
33469
33482
33490
33446
33501
33500
33510
33509
33511
25321
32713
32603
33534
33542
14056
33545
15468
12760
29876
16076
33557
33567
22721
33587
33580
11719
33596
31508
33171
33617
5228
33527
33621
33623
31973
33650
12432
33653
9427
33616
33646
22984
33663
33669
33676
33683
29990
7336
7996
33689
33692
33697
33705
33709
33710
33716
12391
33708
33720
33731
28211
33739
15811
33743
33715
10419
28099
33757
33770
32834
32305
16521
7787
6584
33781
28374
11029
33792
32874
33793
21000
33811
4866
33808
20337
33818
33820
27747
21241
33830
33834
33838
33849
33856
33860
33868
7268
33872
33863
33897
33898
19465
33728
33063
33726
33931
33942
33947
33948
33950
33921
33821
33961
24523
33193
33973
33963
33979
26686
33984
33990
34006
1

21020
39997
39963
36979
40019
40020
23472
40025
40028
18289
24104
38891
40037
36313
40040
19829
40045
40046
26386
26964
40036
40056
17575
40061
40060
40062
35318
40070
40071
40076
31629
40079
40008
39310
40082
7269
40091
40092
40094
40097
9082
40098
38821
40103
40102
40108
40110
19840
40113
40114
40116
40122
40127
40125
40133
37325
39352
40143
40144
26657
40146
24594
40147
40148
35463
40156
40158
40159
40162
40165
40168
40173
40182
40174
40187
35155
31237
40188
40194
40184
40204
40205
23765
40207
10812
40210
23216
40221
27392
40222
40227
40228
40241
40229
40238
40243
37197
39927
40259
40264
40265
38951
39975
40270
40272
40273
40275
32289
40260
36587
38958
40284
40285
40288
26475
30920
40298
40301
40302
30610
40306
9129
40310
27109
28574
40318
40322
40325
40327
36065
40328
40330
40331
38148
37739
40338
1725
25143
28227
40356
40360
40340
31322
12561
40367
10941
40363
36949
37927
31367
40309
40383
39859
40358
40371
13754
40385
40386
17170
40390
40294
40397
40398
40400
40411
40410
23277
40

30359
45088
20134
47714
47716
47446
30511
47666
47734
29593
32572
47719
41070
23402
47752
47694
41838
38592
47768
30682
40813
47771
43051
47641
47790
47791
47802
47811
47763
47816
36971
47826
2367
47839
47524
25720
47764
47846
7007
41548
47853
47856
47862
18518
6889
44396
47876
36207
47879
47869
10351
47883
47872
47632
47526
47481
13386
47355
47904
47787
34947
35024
47926
38541
47687
47928
47940
47898
26212
47955
31439
47905
46609
40303
47968
44210
47911
8005
47998
20452
40750
47717
48007
8540
48012
7664
46136
37730
48022
32558
48024
36870
47938
48031
42328
48034
6414
48038
26952
29809
48043
48050
17105
28555
48047
48059
48058
48060
47860
40258
48070
48071
48066
48075
48079
47684
48081
46271
36597
48086
48088
44921
43461
47814
48097
48098
46718
48099
48103
48104
33270
48106
28214
48095
48113
46649
10437
48119
3558
48129
6670
26516
14520
48091
34012
48135
48137
32821
48146
48151
48156
27412
9602
48162
48163
48166
48169
19956
48181
43851
20415
48186
48188
48190
33139
27841
48193
33368
48

53135
51336
42171
24336
53366
53229
52490
45740
49708
22253
31138
41391
53176
47640
40661
7084
42803
40384
31527
38726
21009
51560
14370
42030
43691
26589
16214
21572
47564
29519
34452
53183
48234
38261
37921
27523
274
49805
51866
50448
32126
51203
15829
52550
28250
40812
44637
53186
36415
16823
40517
52924
50670
18094
46686
13202
51515
32672
53291
51933
40745
18184
44110
8528
51688
38479
41360
50958
31044
16201
27970
45882
34783
44823
17091
21526
21839
20090
32877
28517
40824
53191
22909
10108
43071
17495
52484
52850
37221
18528
53300
38075
37142
51659
49684
32235
19963
12294
34385
47880
10282
51494
5416
36806
34389
18947
37782
52036
13432
35139
23667
25061
5373
49085
42207
51315
47572
45737
46519
20839
52744
49474
4623
41979
16102
46342
32471
50294
49971
48008
51474
45235
32785
34745
41268
6424
46355
47028
43833
46659
45316
27869
8879
18271
20894
49230
52733
49750
44536
34636
14533
34408
42302
41717
43727
49916
51863
40658
52241
33194
20250
50385
39691
22935
46600
9734
11417
27466
40

### Data Prep

Modify the structure of the DF for going into the model

```{python}
{
    user_id: { title : rating }
}
```

In [22]:
# Filters the unique reviewerName for their corresponding title and reviewRatings
rec_prep_data = rec_data.set_index(['user_id', 'book_id']).sort_index()

In [23]:
rec_prep_data.head(50)

goodreads_book_id  \
user_id book_id                      
1       4                     2657   
        10                    1885   
        11                   77203   
        13                    5470   
        16                 2429135   
        22                12232938   
        31                 4667024   
        32                     890   
        33                     930   
        35                     865   
        36                    3636   
        38                   14050   
        40                   19501   
        42                    1934   
        43                   10210   
        46                   43641   
        47                   19063   
        54                      11   
        57                   37435   
        60                    1618   
        66                   18405   
        67                  128029   
        70                  375802   
        72                   11588   
        76                   14935   
        81                    7445   
        85                  370493   
        94                     320   
        95                    5297   
        98                 5060378   
        100                   7244   
        102                  19543   
        103                   7126   
        109                  24280   
        111                  10441   
        115                   2187   
        119                  38447   
        128               11084145   
        131                   4395   
        136                 137791   
        138                  12296   
        140                6892870   
        142                   5043   
        143               18143977   
        148                   2865   
        150                   4989   
        162                  49552   
        171                   6969   
        177                   7144   
        179                 252577   

                                                             title  \
user_id book_id                                                      
1       4                                    To Kill a Mockingbird   
        10                                     Pride and Prejudice   
        11                                         The Kite Runner   
        13                                                    1984   
        16        The Girl with the Dragon Tattoo (Millennium, #1)   
        22                                        The Lovely Bones   
        31                                                The Help   
        32                                         Of Mice and Men   
        33                                     Memoirs of a Geisha   
        35                                           The Alchemist   
        36                               The Giver (The Giver, #1)   
        38                                The Time Traveler's Wife   
        40                                         Eat, Pray, Love   
        42                         Little Women (Little Women, #1)   
        43                                               Jane Eyre   
        46                                     Water for Elephants   
        47                                          The Book Thief   
        54       The Hitchhiker's Guide to the Galaxy (Hitchhik...   
        57                                 The Secret Life of Bees   
        60       The Curious Incident of the Dog in the Night-Time   
        66                                      Gone with the Wind   
        67                                A Thousand Splendid Suns   
        70                         Ender's Game (Ender's Saga, #1)   
        72                            The Shining (The Shining #1)   
        76                                   Sense and Sensibility   
        81                                        The Glass Castle   
        85                                         The Giving Tree   
        94

In [24]:
#Dictionary structure per user is
# {
#     user_id_1: { title : rating,
#                title2 : rating2,
#               ...  
#              },
#     user_id_2: { title : rating,
#                title2 : rating2 
#              },
#     ....
# }
user_entries = (rec_prep_data.groupby('user_id')['title','rating'].apply(lambda x: dict(x.values)).to_dict())

In [25]:
user_entries[123]['The Kite Runner']

5

In [26]:

#Calculates the distance between any given users based on scores they've given similar books
def u2u_euchlid_dist(user_data,uid1,uid2):
    try:
        similar_books = {} 
        for book in user_data[uid1]:
            if book in user_data[uid2]:
                similar_books[book]=1

        #Means that there are no similarities between these two users
        if len(similar_books.keys()) == 0: 
            return 0

        u1 = []
        u2 = []    
        for same_book, value in similar_books.items():        
            u1.append(user_data[uid1][same_book])
            u2.append(user_data[uid2][same_book])        

        euch_dist = np.linalg.norm(np.array(u1) - np.array(u2))
        return euch_dist
    
    except:
        raise Exception("Most likely invalid UID")        

        

# Returns top N people that have the closest Euclidean distance to the specified UID
def top_matches(user_data, uid, n=10, similarity = u2u_euchlid_dist):
    scores = [(similarity(user_data, uid, other_uid), other_uid) for other_uid in user_data if other_uid != uid]
    scores.sort()
    scores.reverse()
    return scores[0:n]


# Gets recommendation using a weighted average of all other users using Euclidean distance score
def get_recommendations(user_data, uid, n=10, similarity = u2u_euchlid_dist):
    book_rec_tracker = {} 
    dist_total = {}
    for other_uid in user_data:
        if other_uid == uid:
            continue
            
        #Euch distance between uid and the current comparison other_uid
        euch_dist = similarity(user_data, uid, other_uid)
        if euch_dist <= 0:
            continue
            
        # Look through books that other_uid has read that uid has not read.
        for book in user_data[other_uid]:
            if book not in user_data[uid] or user_data[uid][book] == 0:

                #Will only set the value if key doesn't exist
                book_rec_tracker.setdefault(book, 0)
                
                #By book we sum over all other_uid distances * the score given for a book by other_uid                     
                book_rec_tracker[book] += user_data[other_uid][book] * euch_dist

                #Will only set the value if key doesn't exist                
                dist_total.setdefault(book, 0)
                
                #By book we sum over all other_uid distances              
                dist_total[book] += euch_dist
                

    #Normalize per book the =  sum(ratings * distances)/sum(distances)
    rankings = [(total/dist_total[book], book) for book, total in book_rec_tracker.items()]
    rankings.sort()
    rankings.reverse()
    return rankings[0:n]

In [27]:
top_matches(user_entries, 1)

[(14.106735979665885, 51577),
 (13.341664064126334, 7871),
 (13.076696830622021, 3641),
 (12.884098726725126, 11257),
 (12.767145334803704, 11793),
 (12.409673645990857, 23722),
 (12.328828005937952, 44171),
 (12.328828005937952, 43435),
 (12.165525060596439, 43433),
 (12.083045973594572, 43400)]

In [28]:
get_recommendations(user_entries, 1)

[(4.803015999635199, 'The Indispensable Calvin and Hobbes'),
 (4.80106008030593, 'The Complete Calvin and Hobbes'),
 (4.7774108092027605,
  'The Authoritative Calvin and Hobbes: A Calvin and Hobbes Treasury'),
 (4.768726494038726,
  "There's Treasure Everywhere: A Calvin and Hobbes Collection"),
 (4.758361071499605, 'ESV Study Bible'),
 (4.757249081057889, 'Harry Potter Boxed Set, Books 1-5 (Harry Potter, #1-5)'),
 (4.755480011355558, "It's a Magical World: A Calvin and Hobbes Collection"),
 (4.753613453152117,
  'Attack of the Deranged Mutant Killer Monster Snow Goons'),
 (4.743329883804913, 'The Calvin and Hobbes Tenth Anniversary Book'),
 (4.734934659062844,
  'The Days Are Just Packed: A Calvin and Hobbes Collection')]